In [593]:
from IPython.core.display_functions import display
from lusidtools.jupyter_tools import toggle_code

"""Term Deposit Valuation

Demonstrates pricing of a Term Deposit Investment.

Attributes
----------
instruments
valuation
lifecycle events
market data store
results store
quotes
"""

toggle_code("Hide docstring")

# Valuing and Rolling Term Deposits
In this notebook, we demonstrate how Term Deposits are handled in LUSID, as well as how to roll term deposits, represent cashflow payements and perform valuations.

# Table of contents


# !! Todo

In [594]:
# Import LUSID libraries
import lusid as lu
import lusid.models as lm
from lusidtools.lpt.lpt import to_date

from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidjam.refreshing_token import RefreshingToken
import lusid.models as models

# Import Libraries
from datetime import datetime, timedelta
import pytz
import pandas as pd
import json
import os

# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:,.4f}'.format)

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = lu.utilities.ApiClientFactory(
        token=RefreshingToken(),
        api_secrets_filename = secrets_path,
        app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.6.11167.0


In [595]:
# Initiate the LUSID APIs required for the notebook
instruments_api = api_factory.build(lu.api.InstrumentsApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lu.AggregationApi)
quotes_api = api_factory.build(lu.QuotesApi)

In [596]:
# Define scopes
scope = "ibor"
portfolio_code = "TermDepositExamplePortfolio"

# 1. Create Portfolio
We must first create portfolio to keep the term depoists in.

In [597]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'TermDepositExamplePortfolio' because it already exists in scope 'ibor'.


# 2. Create Term Deposit Instruments
Next we must create an instrument to represent the Term Deposit. In LUSID the rate of the term deposit must be defined at creation and cannot be adjusted.

In [598]:
# The flow convention defines the payment schedule for the term deposit. This example pays every 6 months.
flow_conventions = lm.FlowConventions(
    currency="USD",
    payment_frequency="6M",
    roll_convention="F",
    day_count_convention="Actual360",
    payment_calendars=[],
    reset_calendars=[],
    settle_days=0,
    reset_days=0,
)

In [599]:
# Define key dates for the term deposits
start_date = datetime(2020, 3, 18, 00, tzinfo=pytz.utc)
maturity_date = datetime(2020, 9, 18, 00, tzinfo=pytz.utc)

In [600]:
# Helper method to create term deposit instrument
def create_td_instrument(
    name,
    identifier,
    start_date,
    maturity_date,
    contract_size,
    flow_convention,
    rate,
    dom_ccy
):
    td_instrument = lm.TermDeposit(
        start_date=start_date,
        maturity_date=maturity_date,
        contract_size=contract_size,
        flow_convention=flow_convention,
        rate=rate,
        dom_ccy=dom_ccy,
        instrument_type="TermDeposit",
        local_vars_configuration=None)

    td_definition = lm.InstrumentDefinition(
        name=name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(identifier)},
        definition=td_instrument
    )

    upsert_request = {identifier: td_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    td_luid = upsert_response.values[identifier]
    print(f"Created instrument with LUID:{td_luid.lusid_instrument_id} and ClientInternal:{identifier}")
    return td_luid.lusid_instrument_id

In [601]:
td_client_a = "TD_Inst_A"
td_luid_a = create_td_instrument(
    "Term Deposit Instrument A",
    td_client_a,
    start_date,
    maturity_date,
    1000,
    flow_conventions,
    0.03,
    "USD"
)

Created instrument with LUID:LUID_00003DET and ClientInternal:TD_Inst_A


# 3. Add Transactions
We will first add an initial cash deposit of $1,000,000 into the portfolio.

In [602]:
txn = lm.TransactionRequest(
transaction_id="txn_000",
type="FundsIn",
instrument_identifiers={"Instrument/default/Currency": "USD"},
transaction_date=(datetime(2020, 3, 10, 00, tzinfo=pytz.utc)).isoformat(),
settlement_date=(datetime(2020, 3, 10, 00, tzinfo=pytz.utc)).isoformat(),
units=1000,
transaction_price=lm.TransactionPrice(price=1,type="Price"),
total_consideration=lm.CurrencyAndAmount(amount=1000000,currency="USD")
)

resp = transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=[txn]
)
print("Successfully upserted transaction:", resp)

Successfully upserted transaction: {'href': 'https://liberty.lusid.com/api/api/transactionportfolios/ibor/TermDepositExamplePortfolio/transactions?asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/portfolios/ibor/TermDepositExamplePortfolio?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ3SGAOL90B:00000004',
            'method': 'GET',
            'relation': 'RequestLogs'}

In [603]:
# Helper method to upsert transactions into LUSID
def upsert_transaction(txn_id, type, luid, client_internal, txn_date, total_consideration):
    txn = lm.TransactionRequest(
    transaction_id=txn_id,
    type=type,
    instrument_identifiers={"Instrument/default/LusidInstrumentId": luid,
                            "Instrument/default/ClientInternal": client_internal},
    transaction_date=  txn_date.isoformat(),
    settlement_date=(txn_date+timedelta(3)).isoformat(),
    units=1000,
    transaction_price=lm.TransactionPrice(price=1,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=total_consideration,currency="USD")
    )

    resp = transaction_portfolios_api.upsert_transactions(
        scope=scope, code=portfolio_code, transaction_request=[txn]
    )
    print("Successfully upserted transaction:", resp)

The next transaction invests $\$1,000,000 into Term Deposit A.
In LUSID this is represented by buying the Term Deposit A Instrument for $\$1,000,000

In [604]:
upsert_transaction("txn001", "Buy", td_luid_a, td_client_a, start_date, 1000000)

Successfully upserted transaction: {'href': 'https://liberty.lusid.com/api/api/transactionportfolios/ibor/TermDepositExamplePortfolio/transactions?asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/portfolios/ibor/TermDepositExamplePortfolio?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2HO0VACLL:0000009F',
            'method': 'GET',
            'relation': 'RequestLogs'}

In [605]:
# Helper method to view the transaction in LUSID up to a specific date
def get_transactions_todate(to_transaction_date):
    get_transactions_reponse = transaction_portfolios_api.get_transactions(
        scope = scope,
        code = portfolio_code,
        to_transaction_date=to_transaction_date
    )
    df = lusid_response_to_data_frame(get_transactions_reponse)
    return df[['transaction_id','type','instrument_identifiers.Instrument/default/ClientInternal','instrument_uid','transaction_date','settlement_date','transaction_price.price','total_consideration.amount','units']]

In [606]:
get_transactions_todate(start_date)

,transaction_id,type,instrument_identifiers.Instrument/default/ClientInternal,instrument_uid,transaction_date,settlement_date,transaction_price.price,total_consideration.amount,units
0,txn_000,FundsIn,NaN,CCY_USD,2020-03-10 00:00:00+00:00,2020-03-10 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
1,txn001,Buy,TD_Inst_A,LUID_00003DET,2020-03-18 00:00:00+00:00,2020-03-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"


# 4. Get Holdings

In [607]:
# Helper method to view holdings in LUSID
def get_holdings(date):
    resp = transaction_portfolios_api.get_holdings(scope=scope,
                                         code=portfolio_code,
                                         property_keys=["Instrument/default/Name"],
                                        effective_at=date)
    df = lusid_response_to_data_frame(resp)
    return df[["instrument_scope","instrument_uid","properties.Instrument/default/Name.value.label_value","units","cost.amount","holding_type_name"]]

In [645]:
get_holdings(start_date+timedelta(4))

,instrument_scope,instrument_uid,properties.Instrument/default/Name.value.label_value,units,cost.amount,holding_type_name
0,default,CCY_USD,USD,"-999,000.0000","-999,000.0000",Balance
1,default,LUID_00003DET,Term Deposit Instrument A,"1,000.0000","1,000,000.0000",Position


# 5. Running a Valuation
In this section we will setup and run a valuation on our portfolio

## 5.1 Setup recipe
To run a valuation in LUSID we must first define a recipe.
The helper method below defines a recipe for a Term Deposit instrument by passing in `instrument_type="TermDeposit"` to the `VendorModelRule`.

In [609]:
# Define a method for creating the recipe
def create_recipe(recipe_code, scope, model):
    # Populate recipe parameters
    recipe = lm.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=lm.MarketContext(
            market_rules=[
                lm.MarketDataKeyRule(
                    key="Equity.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="5D.0D"
                ),
                lm.MarketDataKeyRule(
                    key="FxForwards.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="1Y.0D"
                )
            ],
            options=lm.MarketOptions(
                attempt_to_infer_missing_fx=True,
                default_scope=scope
            )
        ),
        # Set the valuation model - curve with no discounting
        pricing=lm.PricingContext(
            model_rules=[
                lm.VendorModelRule(
                    supplier="Lusid",
                    model_name=model,
                    instrument_type="TermDeposit",
                    parameters="{}",
                )
            ],
            options=lm.PricingOptions(
                produce_separate_result_for_linear_otc_legs=False
            )
        ),
    )

    response = configuration_recipe_api.upsert_configuration_recipe(
                upsert_recipe_request=lm.UpsertRecipeRequest(
                    configuration_recipe=recipe
                )
            )

    return response

LUSID supports SimpleStatic, Discounting and ConstantTimeValueOfMoney pricing models for term deposits. In this notebook we will look at SimpleStatic and ConstantTimeValueOfMoney models.

In [610]:
# Define recipe names
CTVoMRecipe = "TermDepositRecipeCTVoM"
SSRecipe = "TermDepositRecipeSS"

# Create recipes in LUSID
create_recipe(CTVoMRecipe, scope, "ConstantTimeValueOfMoney")
create_recipe(SSRecipe, scope, "SimpleStatic")

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2A6F8Q15L:00000063',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2023, 4, 24, 9, 33, 4, 299540, tzinfo=tzutc())}

In [611]:
# Helper method runs a valuation on the portfolio for a given date and recipe
def perform_valuation(recipe, date):
    # Create valuation request
    valuation_request = lm.ValuationRequest(
        # Choose recipe to use
        recipe_id = lm.ResourceId(scope = scope, code = recipe),
        group_by = ['Instrument/default/LusidInstrumentId'],
        metrics = [
        {"key": "Instrument/default/LusidInstrumentId", "op": "Value"},
        {"key": "Instrument/default/Name", "op": "Value"},              # Reports the friendly name of the underlying instrument
        {"key": "Analytic/default/ValuationDate", "op": "Value"},       # Confirms the valuation date
        {"key": "Valuation/PV", "op": "Sum"}],                          # Calculates cost in GBP (the portfolio currency)
        # Identify portfolio to value
        portfolio_entity_ids = [lm.PortfolioEntityId(scope = scope, code = portfolio_code)],
        # Make date of valuation conditional on effective at date passed into function
        valuation_schedule = lm.ValuationSchedule(effective_at=date),
    )

    # Get portfolio valuation
    val_data = aggregation_api.get_valuation(valuation_request = valuation_request).data

    # Turn valuation response into pandas dataframe
    vals_df = pd.DataFrame(val_data)
    try:
        return vals_df.drop('Aggregation Errors', axis=1)
    except:
        return vals_df

Perform valuation using Constant Time Value of Money Model

In [612]:
perform_valuation(CTVoMRecipe,start_date)

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Valuation/PV)
0,CCY_USD,USD,2020-03-18T00:00:00.0000000+00:00,"-999,000.0000"
1,LUID_00003DET,Term Deposit Instrument A,2020-03-18T00:00:00.0000000+00:00,"1,015,333.3333"


For simple static valuations in LUSID we must provide a quote for the instrument. The date of the quote should be within 5 days of the valuation date. In this example we will set the quote and valuation date at the start of the term deposit.

In [646]:
# Helper method to upsert a quote
# ?? Is this setup correctly
def upsert_quote(luid, date):
    quote_request = {
       "quote_request" : lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=luid,
                    instrument_id_type="LusidInstrumentId",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=date,
            ),
            metric_value=lm.MetricValue(value=1.06, unit="USD"),
        )}

    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(scope=scope, request_body=quote_request)
    return f"Successfully upserted quote for instrument: {response.values['quote_request'].quote_id.quote_series_id.instrument_id} with AsAt:{response.values['quote_request'].as_at}"

In [647]:
upsert_quote(td_luid_a, start_date)

'Successfully upserted quote for instrument: LUID_00003DET with AsAt:2023-04-24 09:37:42.622750+00:00'

In [648]:
perform_valuation(SSRecipe,start_date)

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Valuation/PV)
0,CCY_USD,USD,2020-03-18T00:00:00.0000000+00:00,"-999,000.0000"
1,LUID_00003DET,Term Deposit Instrument A,2020-03-18T00:00:00.0000000+00:00,"1,060,000.0000"


In [616]:
# Helper method - used in the scenarios below - gives a valuation of a specific instrument at a given date
def get_instr_valuation(recipe, date, luid):
    df = pd.DataFrame(perform_valuation(recipe, date))
    instr_val = df[df["Instrument/default/LusidInstrumentId"]==luid]
    return instr_val.iloc[0][3]

## 6. Rolling a Term Deposit
For these scenarios we will always use the Constant Time Value of Money pricing model.
In LUSID we handle rolling a term deposit by selling the term deposit before maturity and buying it back again.
This section will show 2 example scenarios:
1. Rolling with a decrease to principal market value
2. Rolling with an increase to interest


### 6.1 Rolling with a decrease to principal market value

#### 6.1.1 Sell Term Deposit just before maturity
To roll the deposit we first sell the deposit before maturity. We use the instrument valuation calculated on the sale transaction date for the sale price.

In [617]:
def get_cashflow(start_date, maturity_date, recipe):
    resp = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
        scope=scope,
        code=portfolio_code,
        effective_at=maturity_date- timedelta(days=1),
        window_start=start_date,
        window_end=maturity_date + timedelta(days=2),
        recipe_id_scope=scope,
        recipe_id_code=recipe
    )
    display(lusid_response_to_data_frame(resp)[['transaction_id','type','instrument_identifiers.Instrument/default/LusidInstrumentId','units','total_consideration.amount']])
    return resp.values

In [618]:
txn = get_cashflow(start_date, maturity_date, CTVoMRecipe)

#CTVOM adds up future cashflow. ONly one cash flow to cashflow = price.

In [619]:
# Upsert Cashflow movement
transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=txn
)

{'href': 'https://liberty.lusid.com/api/api/transactionportfolios/ibor/TermDepositExamplePortfolio/transactions?asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/portfolios/ibor/TermDepositExamplePortfolio?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2A6F8Q15L:00000068',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {},
 'version': {'a

In [620]:
# Stock out movement
stout_a = lm.TransactionRequest(
    transaction_id="txn002",
    type="StockOut",
    instrument_identifiers={"Instrument/default/LusidInstrumentId": td_luid_a,
                             "Instrument/default/ClientInternal": td_client_a},
    transaction_date=maturity_date.isoformat(),
    settlement_date=(maturity_date+timedelta(3)).isoformat(),
    units=1000,
    transaction_price=lm.TransactionPrice(price=1,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=1000000,currency="USD"),
    exchange_rate=1,
    transaction_currency="USD"
)

transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=[stout_a]
)

{'href': 'https://liberty.lusid.com/api/api/transactionportfolios/ibor/TermDepositExamplePortfolio/transactions?asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/portfolios/ibor/TermDepositExamplePortfolio?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2A6F8Q15L:00000069',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {},
 'version': {'a

### 6.1.2 Buy term deposit back with a increase to market value
In the cell below we will define a new Term Deposit B Instrument which has a start date at the end of the first Term Deposit A and a maturity date 6 months later. In all other aspects it is the same as the first term deposit instrument which was sold above.


In [621]:
# For rolling term deposits we will define new start dates and maturity dates.
roll1_start_date = maturity_date
roll1_maturity_date = datetime(2021,3,18,00,tzinfo=pytz.utc)

pre_maturity_sale_date = maturity_date - timedelta(2)

In [622]:
td_client_b = "TD_Inst_B"
td_luid_b  = create_td_instrument(
    "Term Deposit Instrument B",
    td_client_b,
    roll1_start_date,
    roll1_maturity_date,
    1000,
    flow_conventions,
    0.03,
    "USD"
)

Created instrument with LUID:LUID_00003DEU and ClientInternal:TD_Inst_B


We will now buy the new Term Deposit Instrument B at the same price we initially brought Term Deposit Instrument A.
The is effectively reinvesting the $1,000,000 initially invested in Term Deposit A and the returns into Term Deposit B

In [623]:
inst_a_val = get_instr_valuation(CTVoMRecipe,maturity_date-timedelta(1),td_luid_a)
upsert_transaction("txn003", "Buy", td_luid_b, td_client_b, roll1_start_date, inst_a_val)

Successfully upserted transaction: {'href': 'https://liberty.lusid.com/api/api/transactionportfolios/ibor/TermDepositExamplePortfolio/transactions?asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/portfolios/ibor/TermDepositExamplePortfolio?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2A6F8Q15L:0000006B',
            'method': 'GET',
            'relation': 'RequestLogs'}

In [624]:
get_transactions_todate(roll1_start_date)

,transaction_id,type,instrument_identifiers.Instrument/default/ClientInternal,instrument_uid,transaction_date,settlement_date,transaction_price.price,total_consideration.amount,units
0,txn_000,FundsIn,NaN,CCY_USD,2020-03-10 00:00:00+00:00,2020-03-10 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
1,txn001,Buy,TD_Inst_A,LUID_00003DET,2020-03-18 00:00:00+00:00,2020-03-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
2,txn003,Buy,TD_Inst_B,LUID_00003DEU,2020-09-18 00:00:00+00:00,2020-09-21 00:00:00+00:00,1.0000,"1,015,333.3333","1,000.0000"
3,txn002,StockOut,TD_Inst_A,LUID_00003DET,2020-09-18 00:00:00+00:00,2020-09-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
4,txn001-LUID_00003DET-20200918-Cash-USD-Receive,CashFlow,NaN,LUID_00003DET,2020-09-18 00:00:00+00:00,2020-09-18 00:00:00+00:00,1.0000,"1,015,333.3333","1,015,333.3333"


The table below shows the holdings in the portfolio after the transactions have settled 3 days later.
Here we can see the Term Deposit Instrument A is not longer in the portfolio and has been replaced by Term Deposit Instrument B.
The $15,000 cash balance represents the interest earned from the maturity of Term Deposit A.

In [625]:
get_holdings(roll1_start_date+timedelta(3))

,instrument_scope,instrument_uid,properties.Instrument/default/Name.value.label_value,units,cost.amount,holding_type_name
0,default,CCY_USD,USD,"-999,000.0000","-999,000.0000",Balance
1,default,LUID_00003DEU,Term Deposit Instrument B,"1,000.0000","1,015,333.3300",Position


The valuation below shows the valuation for Term Deposit Instrument B is the same as the valuation was for Term Deposit Instrument A at this at the start of the previous Term Deposit. This is to be expected as both term deposits have the same purchase price, maturity time and rate.

?  The valuation here should be higher

In [626]:
perform_valuation(CTVoMRecipe, roll1_start_date+timedelta(3))

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Valuation/PV)
0,CCY_USD,USD,2020-09-21T00:00:00.0000000+00:00,"-999,000.0000"
1,LUID_00003DEU,Term Deposit Instrument B,2020-09-21T00:00:00.0000000+00:00,"1,015,083.3333"


To perform another simple static evaluation we must upsert a new quote for the new Term Deposit instrument B and the roll1 start date.

In [627]:
upsert_quote(td_luid_b, roll1_start_date+timedelta(3))

'Successfully upserted quote for instrument: LUID_00003DEU with AsAt:2023-04-24 09:33:07.475881+00:00'

In [628]:
perform_valuation(SSRecipe, roll1_start_date+timedelta(3))

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Valuation/PV)
0,CCY_USD,USD,2020-09-21T00:00:00.0000000+00:00,"-999,000.0000"
1,LUID_00003DEU,Term Deposit Instrument B,2020-09-21T00:00:00.0000000+00:00,"1,050,250.0000"


## 6.2 Rolling with an increase to interest
### 6.2.1 Sell Term Deposit just before maturity
As in the previous rolling example, we first sell the deposit before maturity. We use the instrument valuation calculated on the sale transaction date for the sale price.

In [629]:
# For rolling term deposits we will define new start dates and maturity dates.
roll2_maturity_date = datetime(2021,9,18,00,tzinfo=pytz.utc)

In [630]:
perform_valuation(CTVoMRecipe, roll1_maturity_date-timedelta(2))

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Valuation/PV)
0,CCY_USD,USD,2021-03-16T00:00:00.0000000+00:00,"-999,000.0000"
1,LUID_00003DEU,Term Deposit Instrument B,2021-03-16T00:00:00.0000000+00:00,"1,015,083.3333"


In [631]:
#?? remove this
txn = get_cashflow(roll1_start_date+timedelta(3), roll1_maturity_date, CTVoMRecipe)

In [632]:
# Upsert Cashflow movement
transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=txn
)

{'href': 'https://liberty.lusid.com/api/api/transactionportfolios/ibor/TermDepositExamplePortfolio/transactions?asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/portfolios/ibor/TermDepositExamplePortfolio?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2HO0VACLL:000000A2',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {},
 'version': {'a

In [633]:
# This time we will reinvest all the returns from the cashflow
upsert_transaction("txn004", "StockOut",td_luid_b, td_client_b,roll1_maturity_date,1000000)

Successfully upserted transaction: {'href': 'https://liberty.lusid.com/api/api/transactionportfolios/ibor/TermDepositExamplePortfolio/transactions?asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/portfolios/ibor/TermDepositExamplePortfolio?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2A6F8Q15L:00000072',
            'method': 'GET',
            'relation': 'RequestLogs'}

In [634]:
get_transactions_todate(roll1_maturity_date)

,transaction_id,type,instrument_identifiers.Instrument/default/ClientInternal,instrument_uid,transaction_date,settlement_date,transaction_price.price,total_consideration.amount,units
0,txn_000,FundsIn,NaN,CCY_USD,2020-03-10 00:00:00+00:00,2020-03-10 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
1,txn001,Buy,TD_Inst_A,LUID_00003DET,2020-03-18 00:00:00+00:00,2020-03-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
2,txn003,Buy,TD_Inst_B,LUID_00003DEU,2020-09-18 00:00:00+00:00,2020-09-21 00:00:00+00:00,1.0000,"1,015,333.3333","1,000.0000"
3,txn002,StockOut,TD_Inst_A,LUID_00003DET,2020-09-18 00:00:00+00:00,2020-09-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
4,txn001-LUID_00003DET-20200918-Cash-USD-Receive,CashFlow,NaN,LUID_00003DET,2020-09-18 00:00:00+00:00,2020-09-18 00:00:00+00:00,1.0000,"1,015,333.3333","1,015,333.3333"
5,txn003-LUID_00003DEU-20210318-Cash-USD-Receive,CashFlow,NaN,LUID_00003DEU,2021-03-18 00:00:00+00:00,2021-03-18 00:00:00+00:00,1.0000,"1,015,083.3333","1,015,083.3333"
6,txn004,StockOut,TD_Inst_B,LUID_00003DEU,2021-03-18 00:00:00+00:00,2021-03-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
7,txn005,Buy,TD_EXAMPLE_C,LUID_00003DEP,2021-03-18 00:00:00+00:00,2021-03-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"


In [635]:
get_holdings(roll1_maturity_date+timedelta(3))

,instrument_scope,instrument_uid,properties.Instrument/default/Name.value.label_value,units,cost.amount,holding_type_name
0,default,CCY_USD,USD,"-983,916.6667","-983,916.6700",Balance
1,default,LUID_00003DEP,TD Example Instrument C,"1,000.0000","1,000,000.0000",Position


### 6.2.2 Buy term deposit back with an Increased Rate
We will create a new instrument below with a higher rate of 0.05 -> 5% (the original Term Deposit had a rate of 3%).

In [636]:
# Define a new instrument with a higher rate
td_client_c = "TD_EXAMPLE_C"
td_luid_c = create_td_instrument(
    "TD Example Instrument C",
    td_client_c,
    roll1_maturity_date,
    roll2_maturity_date,
    1000,
    flow_conventions,
    0.05,
    "GBP"
)

Created instrument with LUID:LUID_00003DEP and ClientInternal:TD_EXAMPLE_C


This new instrument will be purchased at the same price the previous term deposit was sold at.

In [637]:
upsert_transaction("txn005","Buy",td_luid_c, td_client_c, roll1_maturity_date, 1000000)

Successfully upserted transaction: {'href': 'https://liberty.lusid.com/api/api/transactionportfolios/ibor/TermDepositExamplePortfolio/transactions?asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/portfolios/ibor/TermDepositExamplePortfolio?effectiveAt=2010-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-04-24T07%3A37%3A16.1158760%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2A6F8Q15L:00000075',
            'method': 'GET',
            'relation': 'RequestLogs'}

In [638]:
get_transactions_todate(roll1_maturity_date + timedelta(days=3))

,transaction_id,type,instrument_identifiers.Instrument/default/ClientInternal,instrument_uid,transaction_date,settlement_date,transaction_price.price,total_consideration.amount,units
0,txn_000,FundsIn,NaN,CCY_USD,2020-03-10 00:00:00+00:00,2020-03-10 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
1,txn001,Buy,TD_Inst_A,LUID_00003DET,2020-03-18 00:00:00+00:00,2020-03-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
2,txn003,Buy,TD_Inst_B,LUID_00003DEU,2020-09-18 00:00:00+00:00,2020-09-21 00:00:00+00:00,1.0000,"1,015,333.3333","1,000.0000"
3,txn002,StockOut,TD_Inst_A,LUID_00003DET,2020-09-18 00:00:00+00:00,2020-09-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
4,txn001-LUID_00003DET-20200918-Cash-USD-Receive,CashFlow,NaN,LUID_00003DET,2020-09-18 00:00:00+00:00,2020-09-18 00:00:00+00:00,1.0000,"1,015,333.3333","1,015,333.3333"
5,txn003-LUID_00003DEU-20210318-Cash-USD-Receive,CashFlow,NaN,LUID_00003DEU,2021-03-18 00:00:00+00:00,2021-03-18 00:00:00+00:00,1.0000,"1,015,083.3333","1,015,083.3333"
6,txn004,StockOut,TD_Inst_B,LUID_00003DEU,2021-03-18 00:00:00+00:00,2021-03-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"
7,txn005,Buy,TD_EXAMPLE_C,LUID_00003DEP,2021-03-18 00:00:00+00:00,2021-03-21 00:00:00+00:00,1.0000,"1,000,000.0000","1,000.0000"


The holdings shown below reflect the $1,000 price difference in the sell price of Term Deposit A and the buy price of Term Deposit B, in the USD cash holding.

In [639]:
get_holdings(roll1_maturity_date + timedelta(days=3))

,instrument_scope,instrument_uid,properties.Instrument/default/Name.value.label_value,units,cost.amount,holding_type_name
0,default,CCY_USD,USD,"-983,916.6667","-983,916.6700",Balance
1,default,LUID_00003DEP,TD Example Instrument C,"1,000.0000","1,000,000.0000",Position


In [640]:
perform_valuation(CTVoMRecipe, roll1_maturity_date+timedelta(3))

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Valuation/PV)
0,CCY_USD,USD,2021-03-21T00:00:00.0000000+00:00,"-983,916.6667"
1,LUID_00003DEP,TD Example Instrument C,2021-03-21T00:00:00.0000000+00:00,"1,025,555.5556"


In [641]:
upsert_quote(td_luid_c, rolUl1_maturity_date + timedelta(3))

'Successfully upserted quote for instrument: LUID_00003DEP with AsAt:2023-04-24 09:33:10.604739+00:00'

In [642]:
perform_valuation(SSRecipe, roll1_maturity_date+timedelta(3))

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Valuation/PV)
0,CCY_USD,USD,2021-03-21T00:00:00.0000000+00:00,"-983,916.6667"
1,LUID_00003DEP,TD Example Instrument C,2021-03-21T00:00:00.0000000+00:00,"1,050,416.6667"


In [642]:
# AP to make changes to term deposits and cash flows. Will be 2 cash flows.

### 5.2 Valuations using CTVoM

? How do I feed valuation price into sale price
? Should valuation prices change with time
? Does the second purchase after the roll need to have a later maturity

In [643]:
# # Book a sell transaction against the term deposit
# preMaturityValuation = instA_valuation(rolling_date)
#
# sell_td = lm.TransactionRequest(
#     transaction_id="txn003",
#     type="Sell",
#     instrument_identifiers={"Instrument/default/ClientInternal": "TD_EXAMPLE_A"},
#     transaction_date=rolling_date.isoformat(),
#     settlement_date=(rolling_date + timedelta(days=30)).isoformat(),
#     units=75,
#     transaction_price=lm.TransactionPrice(price=0,type="Price"),
#     total_consideration=lm.CurrencyAndAmount(amount=preMaturityValuation,currency="USD"),
#     exchange_rate=1,
#     transaction_currency="USD"
# )
#
# response = transaction_portfolios_api.upsert_transactions(scope=scope,
#                                                     code=portfolio_code,
#                                                     transaction_request=[sell_td])
# response

In [644]:
# # Buy TD back with lower market value
#
# sell_td = lm.TransactionRequest(
#     transaction_id="txn004",
#     type="Sell",
#     instrument_identifiers={"Instrument/default/ClientInternal": "TD_EXAMPLE_A"},
#     transaction_date=rolling_date.isoformat(),
#     settlement_date=(rolling_date + timedelta(days=30)).isoformat(),
#     units=75,
#     transaction_price=lm.TransactionPrice(price=0,type="Price"),
#     total_consideration=lm.CurrencyAndAmount(amount=preMaturityValuation,currency="USD"),
#     exchange_rate=1,
#     transaction_currency="USD"
# )